# What do Principal Components Actually do Mathematically?
I have recently taken an interest in PCA after watching Professor Gilbert Strang’s [SVD lecture](https://www.youtube.com/watch?v=rYz83XPxiZo). I must have watched at least 15 other videos and read 7 different blog posts on PCA since. They are all very excellent resources, but I found myself somewhat unsatisfied. What they do a lot is teaching us the following:
- What the PCA promise is;
- Why that promise is very useful in Data Science; and
- How to extract these principal components. (Although I don't agree with how some of them do it by applying SVD on the covariance matrix, that can be saved for another post.)

Some of them go the extra mile to show us graphically or logically, how the promise is being fulfilled by the principal components:
- Graphically, a transformed vector can be shown to be still clustered with its original group in a plot; and
- Logically, a proof can be expressed in mathematical symbols.

## Objective
My mind was convinced, but my heart was still not. To me, the graphic approach does not provide a visual effect that is striking enough. The logic approach, on the other hand, does not even spend enough effort to explain what precisely it's trying to prove. Therefore, the objective of this post is to improve both of these 2 areas - to provide a more striking visual and to establish a more precise goal for our mathematical proof.

## Prerequisites
This post is for you if:
- You have at least a slight idea of what the PCA promise is;
- You have a decent understanding of what the covariance matrix is about;
- You have a solid foundation in linear algebra, e.g., comfortable with the concept of matrices being transformations that rotate and/or stretch spaces; and
- Your heart is longing to discover the principal components, instead of being told what they are!

## How to Choose P?
After hearing my dissatisfaction, my friend [Calvin](https://calvinfeng.github.io/) recommended this paper by Jonathon Shlens - [A Tutorial on Principal Component Analysis](https://arxiv.org/pdf/1404.1100.pdf) to me. It is by far the best resource I have come across on PCA. However, it's also a bit lengthier than your typical blog post, so the remainder of this post will focus on section 5 of the paper. In there, Jonathon immediately establishes the following goal:
> The [original] dataset is $X$, an $m × n$ matrix.<br>
> Find some orthonormal matrix $P$ in $Y = PX$ such that $C_Y \equiv \frac{1}{n}YY^T$ is a diagonal matrix.[1]<br>
> The rows of $P$ shall be principal components of $X$.

As you might have noticed, $C_Y$ here is the covariance matrix of our rotated dataset $Y$. Why do we want $C_Y$ to be diagonal? Before we answer this question, let’s generate a dataset $X$ consisting of 4 features with some random values.

In [130]:
import numpy as np
feature_num, sample_num = 4, 6
x = np.random.rand(feature_num, sample_num)

def covariance_matrix(dataset):
    sample_count = dataset.shape[1]
    return (dataset @ dataset.transpose() / sample_count).round(15)

In [141]:
c_x = covariance_matrix(x)
_, p = np.linalg.eig(c_x)
y = p.transpose() @ x
c_y = covariance_matrix(y)

In [143]:
from IPython.display import display
import pandas as pd

display(pd.DataFrame.from_records(x, columns=['sample{}'.format(num) for num in range(sample_num)], index=['feat_{}'.format(num) for num in 'abcd']))
display(pd.DataFrame.from_records(y, columns=['sample{}'.format(num) for num in range(sample_num)], index=['feat_{}'.format(num) for num in 'efgh']))
display(pd.DataFrame.from_records(c_x, columns=['feat_{}'.format(num) for num in 'abcd'], index=['feat__{}'.format(num) for num in 'abcd']))
display(pd.DataFrame.from_records(c_y, columns=['feat_{}'.format(num) for num in 'efgh'], index=['feat__{}'.format(num) for num in 'efgh']))

,sample0,sample1,sample2,sample3,sample4,sample5
feat_a,0.364316,0.369955,0.414107,0.398438,0.273329,0.974257
feat_b,0.233349,0.781982,0.420029,0.308131,0.333717,0.362706
feat_c,0.089461,0.010197,0.128515,0.376316,0.229315,0.730119
feat_d,0.381779,0.965491,0.635578,0.879495,0.542847,0.051483


,sample0,sample1,sample2,sample3,sample4,sample5
feat_e,0.573571,1.200266,0.866370,1.037611,0.721114,0.905122
feat_f,0.006382,-0.446941,-0.115716,-0.114439,-0.075257,0.890546
feat_g,-0.105567,0.040343,-0.055121,0.001167,0.057684,0.018866
feat_h,-0.027708,-0.196512,-0.036851,0.281227,0.065940,-0.061503


,feat_a,feat_b,feat_c,feat_d
feat__a,0.270619,0.185934,0.168921,0.218072
feat__b,0.185934,0.196706,0.090022,0.263646
feat__c,0.168921,0.090022,0.125316,0.103120
feat__d,0.218072,0.263646,0.103120,0.425455


,feat_e,feat_f,feat_g,feat_h
feat__e,0.822684,0.000000,-0.000000,-0.000000
feat__f,0.000000,0.170837,0.000000,0.000000
feat__g,-0.000000,0.000000,0.003249,-0.000000
feat__h,-0.000000,0.000000,-0.000000,0.021327


[1]: The reason orthonormality is part of the goal is that we do not want to do anything more than rotating $X$. We do not want to modify $X$. We only want to re-express $X$ by carefully choosing a change of basis.